import pickle
import statistics
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Imports

In [1]:
import folium
from collections import Counter

In [16]:
stations_df = pd.read_csv("./saved_data/final_station_df.csv", index_col = 0)

<IPython.core.display.Javascript object>

In [25]:
non_unique_stations_df = pd.read_csv("./saved_data/non_unique_mta_stations.csv", index_col=0)

<IPython.core.display.Javascript object>

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.manifold import TSNE
import pandas as pd
from pyspark import SparkContext
import statistics
import altair as alt
# ^^^ pyforest auto-imports - don't write above this line
# Finding Routes

In [3]:
stop_times_df = pd.read_csv("./gtfs_data/stop_times.txt")

<IPython.core.display.Javascript object>

In [4]:
stop_times_df.shape

(535099, 9)

### Removing unnecessary columns

In [5]:
null_columns2=stop_times_df.columns[stop_times_df.isnull().any()]
stop_times_df[null_columns2].isnull().sum()

stop_headsign          535099
shape_dist_traveled    535099
dtype: int64

In [6]:
stop_times_df.drop(columns = ['stop_headsign', 'shape_dist_traveled'], inplace=True)

In [7]:
print(stop_times_df.pickup_type.nunique())
print(stop_times_df.drop_off_type.nunique())

2
2


In [261]:
# I think I need to use the stop_sequence and the stop_id to map them out 
## A lot of suspiciously round numbers
stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,0,0
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,0,0
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,0,0
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,0,0
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,0,0
...,...,...,...,...,...,...,...
535094,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:06:00,24:06:00,L25S,14,0,0
535095,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:07:00,24:07:00,L26S,15,0,0
535096,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:08:30,24:08:30,L27S,16,0,0
535097,BFA19SUPP-L047-Weekday-99_142400_L..S10R,24:10:30,24:10:30,L28S,17,0,0


## Getting Connections to Each Stop

In [214]:
# vectorizing the arrays
stop_id_order_array = np.array(stop_times_df['stop_id'])
# station_id_array = np.array(stations_df['station_id'])
# stop_id_stations_array = np.array(non_unique_stations_df['stop_id']) # will combine duplicate station_ids later
stop_id_order_array_no_ns = np.array([x[:-1] for x in stop_times_df['stop_id']]) # all stop_ids end in n or s

In [33]:
stop_id_connections_dict = {key:None for key in non_unique_stations_df['stop_id']}

In [146]:
def get_connections_to_a_stop(the_stop, stop_times_stop_id=stop_id_order_array_no_ns, df=stop_times_df):
    # make list of stop_ids that are bordering a certain stop
    # https://stackoverflow.com/questions/432112/is-there-a-numpy-function-to-return-the-first-index-of-something-in-an-array
    indices_of_the_stop = np.where(stop_times_stop_id == the_stop)
    all_connections = []
    for idx in indices_of_the_stop[0]:
        if idx > 0:
            prev_row = df.iloc[idx-1]
            if prev_row['trip_id'] == df['trip_id'][idx]:
                all_connections.append(prev_row['stop_id'][:-1])
        elif idx < df.shape[0]:
            next_row = df.iloc[idx+1]
            if next_row['trip_id'] == df['trip_id'][idx]:
                all_connections.append(next_row['stop_id'][:-1])
    connections_to_the_stop = set(all_connections)
    if the_stop in connections_to_the_stop:
        connections_to_the_stop.remove(the_stop)
    return connections_to_the_stop

In [155]:
def add_connections_to_dict(the_stop, the_dict=stop_id_connections_dict, func = get_connections_to_a_stop):
    if the_dict.get(the_stop) == None:
        connections_to_the_stop = get_connections_to_a_stop(the_stop)
        the_dict[the_stop] = connections_to_the_stop
    return the_dict

In [162]:
# took about a minute to run
for stop in non_unique_stations_df['stop_id']:
    stop_id_connections_dict = add_connections_to_dict(the_stop=stop)

In [213]:
# stop_id_connections_dict

#### Saving connections dict

In [180]:
# with open("./saved_data/all_connections_dict.pickle", "wb+") as f:
#     pickle.dump(stop_id_connections_dict, f)

<IPython.core.display.Javascript object>

### Which station has the most direct connections

In [177]:
most_connections = 0
for key, value in stop_id_connections_dict.items():
    if len(value) > most_connections:
        most_connections = len(value)
print(most_connections)

5


In [176]:
most_connections_station = ""
most_connections_station_common_name = []
for key, value in stop_id_connections_dict.items():
    if len(value) >= 5:
        most_connections_station = key
        index = non_unique_stations_df['stop_id'] == key
        common_name = list(non_unique_stations_df['stop_name'][index])
        common_name.append(f"officially known as {key}")
        most_connections_station_common_name.append(common_name)
for x in most_connections_station_common_name:
    print(x)

['Franklin Av', 'officially known as 239']
['125 St', 'officially known as 621']
['125 St', 'officially known as A15']
['59 St - Columbus Circle', 'officially known as A24']
['Hoyt - Schermerhorn Sts', 'officially known as A42']
['Kew Gardens - Union Tpke', 'officially known as F06']
['Jackson Hts - Roosevelt Av', 'officially known as G14']
['Myrtle Av', 'officially known as M11']
['DeKalb Av', 'officially known as R30']
['Atlantic Av - Barclays Ctr', 'officially known as R31']
['36 St', 'officially known as R36']


### Putting Connections into a dataframe

#### Get All Combinations of Stops

In [188]:
# only need one combination of each pair (not each permutation)
## e.g.: 101 <-> 103 should have same amount of time between them
permutation_list = []
for key, value in stop_id_connections_dict.items():
    for item in value:
        permutation = [key, item]
        if [item, key] not in permutation_list:
            permutation_list.append(permutation)

In [189]:
# there are 551 distinct station to station pairs in the NYC subway system
len(permutation_list)

551

In [246]:
# permutation_list

#### Making sure second avenue subway cars are in list

In [196]:
# new 96 st station has the id 625 which is in the dict! 
# for idx, x in enumerate(non_unique_stations_df['stop_name']):
#     if '96' in x:
#         print(non_unique_stations_df.iloc[idx])

#### Putting combos in df

In [209]:
start_station = [x[0] for x in permutation_list]
end_station = [x[1] for x in permutation_list]

In [450]:
edge_list_df = pd.DataFrame(data=[start_station, end_station]).T

<IPython.core.display.Javascript object>

In [451]:
edge_list_df.columns = ['node1', 'node2']
edge_list_df['trail'] = 'nyc subway' # making this a single trail for simplicity's sake
edge_list_df['color'] = 'red'

## Getting Distance Between Each Connection

In [216]:
# will add this connection "distance" to the distance column in the edge_list_df
## first: are the time columns in stop_times_df timestamps? can I subtract them?
stop_times_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,0,0
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,0,0
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,0,0
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,0,0
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,0,0


In [219]:
type(stop_times_df['arrival_time'][0])

str

In [221]:
neg_times = []
over_24 = []
for x in stop_times_df['arrival_time']:
    if '-' in x:
        neg_times.append(x)
    elif '24' in x:
        over_24.append(x)

In [222]:
len(neg_times)

0

In [223]:
len(over_24)

20019

In [224]:
arrival_times_datetime = pd.to_datetime(stop_times_df['arrival_time'], errors = 'coerce', infer_datetime_format=True)

<IPython.core.display.Javascript object>

In [226]:
type(arrival_times_datetime[0])

pandas._libs.tslibs.timestamps.Timestamp

In [227]:
arrival_times_datetime[0]

Timestamp('2020-07-22 00:06:00')

In [231]:
arrival_times_timedelta = pd.to_timedelta(arrival_times_datetime, unit = 'seconds', errors = 'coerce')

<IPython.core.display.Javascript object>

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/util/_decorators.py:208: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  return func(*args, **kwargs)


In [245]:
arrival_times_timedelta.isnull().sum()

18044

In [236]:
type(arrival_times_timedelta[1])

pandas._libs.tslibs.timedeltas.Timedelta

In [237]:
pd.Timedelta(arrival_times_timedelta[0], arrival_times_timedelta[1])

<IPython.core.display.Javascript object>

Timedelta('18465 days 00:06:00')

In [239]:
x = arrival_times_timedelta[1] - arrival_times_timedelta[0]

In [240]:
print(x.seconds)

90


In [452]:
stop_times_df['arrival_times_timedelta'] = arrival_times_timedelta
edge_list_df['distance'] = 0 # empty column where I will add the distance between nodes

### Checking if there is a difference in time between stops depending on day of week

In [308]:
indices_of_the_stop = np.where(stop_id_order_array_no_ns == non_unique_stations_df['stop_id'][0])

In [301]:
sat = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1038-Saturday")]
sun = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1037-Sunday")]
weekday = stop_times_df[stop_times_df.trip_id.str.contains("AFA19GEN-1087-Weekday")]

In [302]:
# looks like it's the same 1:30 each way at all times
sun_and_first_2_stops = sun[(sun.stop_id.str.contains("101") | (sun.stop_id.str.contains("103")))]
sat_and_first_2_stops = sat[(sat.stop_id.str.contains("101") | (sat.stop_id.str.contains("103")))]
weekday_and_first_2_stops = weekday[(weekday.stop_id.str.contains("101") | (weekday.stop_id.str.contains("103")))]

### Finding "distance" between each node

In [453]:
edge_list_df

,node1,node2,trail,color,distance
0,101,103,nyc subway,red,0
1,103,104,nyc subway,red,0
2,104,106,nyc subway,red,0
3,106,107,nyc subway,red,0
4,107,108,nyc subway,red,0
...,...,...,...,...,...
546,R42,R43,nyc subway,red,0
547,R43,R44,nyc subway,red,0
548,R44,R45,nyc subway,red,0
549,S01,S03,nyc subway,red,0


- choose node
- look for node
- look for second node (immediately following)
- compute timedelta between them
- add timedelta to correct row
- move onto next row in df

In [454]:
def find_edge_distance(row_of_edge_list_df, df=edge_list_df, stop_times_df=stop_times_df):
    null_list = stop_times_df['arrival_times_timedelta'].isnull()
    node_location = row_of_edge_list_df
    if (df['distance'][node_location] == 0):
        node1 = df['node1'][node_location]
        node2 = df['node2'][node_location]
        indices_of_the_stop = np.where(stop_id_order_array_no_ns == node1)
        time_delta = 0
        for idx in indices_of_the_stop[0]:
            next_row = stop_times_df.iloc[idx+1]
            stop_id_dest = next_row['stop_id'][:-1]
            if stop_id_dest == node2:
                if (null_list[idx+1] == False) & (null_list[idx] == False):
                    if stop_times_df['trip_id'][idx+1] == stop_times_df['trip_id'][idx]:
                        time_delta = stop_times_df['arrival_times_timedelta'][idx+1] - stop_times_df['arrival_times_timedelta'][idx]
                        df['distance'][node_location] = time_delta.seconds
                        break
    return df

In [455]:
for x in range(0, len(edge_list_df)):
    edge_list_df = find_edge_distance(x)

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [456]:
# no nulls!
edge_list_df['distance'].isnull().sum()

0

In [457]:
edge_list_df['distance'].min()

0

In [ ]:
# fix the duplicate stations (at least in node1 and node2 lists)
## might have to make all nodes with only one connection required

### Fixing the duplicate stations

In [397]:
duplicate_stations = [x for x in stations_df['station_id'] if len(x) > 4]

In [425]:
duplicate_stations

['718_R09', 'A13_D13', 'A32_D20', 'D43_N12', 'H01_H02']

In [418]:
duplicate_stations_ind1 = [x.split("_") for x in duplicate_stations]
duplicate_stations_ind = list(np.array(duplicate_stations_ind1).flat)

In [426]:
stations_w_station_id = {key:None for key in duplicate_stations_ind}
duplicate_stations_ind

['718', 'R09', 'A13', 'D13', 'A32', 'D20', 'D43', 'N12', 'H01', 'H02']

In [427]:
counter = 0
keys_list = list(stations_w_station_id.keys())
for item in keys_list:
    for thing in duplicate_stations:
        if item in thing:
            stations_w_station_id[item] = thing

In [420]:
# gathering the examples from the edge_list
duplicate_edge_df = edge_list_df[(edge_list_df['node1'].isin(duplicate_stations_ind))|
                                 (edge_list_df['node2'].isin(duplicate_stations_ind))]

In [422]:
len(duplicate_edge_df)

22

In [431]:
duplicate_edge_df

,node1,node2,trail,color,distance
183,712,718,nyc subway,red,330
187,716,718,nyc subway,red,120
188,718,719,nyc subway,red,90
208,A14,D13,nyc subway,red,180
211,A15,D13,nyc subway,red,210
228,A31,A32,nyc subway,red,120
229,A32,A34,nyc subway,red,150
230,A32,A33,nyc subway,red,90
264,A61,H01,nyc subway,red,0
286,B23,D43,nyc subway,red,420


In [435]:
edge_list_df.replace(to_replace=stations_w_station_id, inplace=True)

In [441]:
# is actually legit b/c an express train makes this route
edge_list_df.distance.max()

780

In [ ]:
duplicate_edge_df[duplicate_edge_df['']]

### Fixing stations with errors in distance

In [484]:
edge_list_df.distance.sort_values()

409      0
110      0
264      0
48       0
408      0
      ... 
375    570
90     630
502    660
313    660
55     780
Name: distance, Length: 551, dtype: int64

In [458]:
len(edge_list_df[edge_list_df['distance'] == 0])

5

In [460]:
# 48, 110 looks like a mistake
# 408 share the same parent station
# 264, 409 is b/c of a northbound v southbound issue
edge_list_df[edge_list_df['distance'] == 0]

,node1,node2,trail,color,distance
48,208,213,nyc subway,red,0
110,409,413,nyc subway,red,0
264,A61,H01,nyc subway,red,0
408,H01,H02,nyc subway,red,0
409,H02,A61,nyc subway,red,0


In [488]:
edge_list_df[edge_list_df['node1'] == '208']

,node1,node2,trail,color,distance
48,208,213,nyc subway,red,0
49,208,209,nyc subway,red,90


In [480]:
edge_list_df[edge_list_df['node2'] == 'A61']

,node1,node2,trail,color,distance
262,A60,A61,nyc subway,red,90
409,H02,A61,nyc subway,red,0


In [489]:
edge_list_df[edge_list_df['node1'] == '208']

,node1,node2,trail,color,distance
48,208,213,nyc subway,red,0
49,208,209,nyc subway,red,90


In [503]:
non_unique_stations_df[non_unique_stations_df['stop_id'] == 'A61']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
681,A61,Rockaway Blvd,40.680429,-73.843853,1,NaN


In [495]:
non_unique_stations_df[non_unique_stations_df['stop_id'] == 'H02']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
1047,H02,Aqueduct - N Conduit Av,40.668234,-73.834058,1,NaN


#### Dropping mistaken connections

In [493]:
edge_list_df.drop(index=[408, 48, 110], inplace=True)

#### Manually finding values for the ones still at 0 

- 4 minutes between Aqueduct - N Conduit Av and Rockaway Blvd

In [496]:
# indices_of_the_stop = np.where(stop_id_order_array_no_ns == 'H02')
# indices_of_the_stop[0][0:5]
# stop_times_df.iloc[223345:223350]

In [510]:
edge_list_df['distance'][264] = 4 * 60
edge_list_df['distance'][409] = 4 * 60

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Adding "required" edges to df
only required for the rural postman problem

In [511]:
# only 1 edge list 
one_connection_list = []
for key, value in stop_id_connections_dict.items():
    if len(value) == 1:
        one_connection_list.append(key)

In [513]:
len(one_connection_list)

29

In [ ]:
edge_list_df['required']

In [518]:
# for x in one_connection_list:
#     for node in edge_list_df['node1']:
#         if node == x:
#             print("yay")

## Saving edge_list_df

In [520]:
edge_list_df.reset_index(drop=True, inplace=True)

In [522]:
# edge_list_df.to_csv("./saved_data/edge_list_df_no_req.csv")

In [523]:
edge_list_df

,node1,node2,trail,color,distance
0,101,103,nyc subway,red,90
1,103,104,nyc subway,red,90
2,104,106,nyc subway,red,90
3,106,107,nyc subway,red,90
4,107,108,nyc subway,red,60
...,...,...,...,...,...
543,R42,R43,nyc subway,red,60
544,R43,R44,nyc subway,red,120
545,R44,R45,nyc subway,red,300
546,S01,S03,nyc subway,red,120
